# Inverse design with Keras Neural Networks

### Level: Intermediate

In this notebook, we show how to achieve the inverse desing of the input parameters to provide to an ANN in order to achieve an specific output. 

First, we create a basic neural network with optimized hiperparameters [1,2]. Then, we adress the optimization process between the desired output and the ANN output as a function of the ANN inputs.

References and additional documentation at the end of this notebook.

#### Dependencies

In [1]:
import keras
import keras_tuner as kt
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2024-11-26 16:39:22.834972: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-26 16:39:22.835346: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 16:39:22.837847: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 16:39:22.844029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732635562.854142  188303 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732635562.85

In [2]:
# fix random seeds
seed = 0
np_rng = np.random.default_rng(seed)  # [3]
keras.utils.set_random_seed(seed)

#### Data

Load train data and split into train/test groups.

In [3]:
try:
    X = pd.read_parquet("features.parquet")
    y = pd.read_parquet("targets.parquet")
except FileNotFoundError:
    # For didactic purposes, we define a set of data with binary representation
    # of 0 to 3 as input and the decimal representation as output.
    binary_digits = 2

    # We add non-integer inputs as the optimizer will feed our NN with them, so
    # we want our model to be ready for such inputs training it with them.
    n_samples = 1000
    lower_bound = 0
    higher_bound = 1
    X = pd.DataFrame(
        np_rng.uniform(
            low=lower_bound, high=higher_bound, size=(n_samples, binary_digits)
        )
    )

    # We transfor previous data into its decimal number. Don't worry about it!
    # ONly if you are curiorious, this is the process:
    # * We transform one by one each row of X, which stores a single number with
    #   its digits in each column (columns 0 : 1, column 1 : 1), with the `int`
    #   operation, which converts a number with the specified base, in our case,
    #   2, into an integer number (int('11', 2) == 3).
    # * To transform the binary number with `int` function to the decimal
    #   number, we previously round and transform to int the decimal numbers of
    #   each digit (column 0 : 0.891 -> 1, column 1 : 0.55 -> 1) and store all
    #   those numbers into a single string with `"".join(map(str, x))`
    #   (column 0 : 1, column 1 : 1 -> '1, 1')
    y = X.apply(
        lambda number: int("".join(map(str, (np.round(number)).astype(int))), 2), axis=1
    )

    # we set a fixed random state for reproducibility and teaching purposes
    X, y = shuffle(X, y, random_state=seed)

# we set a fixed random state for reproducibility and teaching purposes,
# but our results have to be consistent across multiple seeds to be relevant
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.9, test_size=0.1, random_state=seed
)

# shape of input features and output predictions
features_shape = X_train.iloc[0].shape
target_shape = y_train.iloc[0].shape if bool(y_train.iloc[0].shape) else 1

#### Some parameters

In [4]:
# model optimizer
loss = "mae"
lr = {"min_value": 1e-4, "max_value": 1e-2}  # values from 0.0001 to 0.01
metrics = ["accuracy"]

# training
epochs = 2000
batch_size = 16

# hp tuner
max_trials = 50

# early stopping
monitor = "val_loss"
patience = 20

## Neural Network model

Hyperparameter optimization and NN model training.

Optimization regarding epoch number is skipped as early stopping is considered a sufficient method for achieving it.

In [5]:
def model_builder(hp):
    """Build a neural network model."""
    input_layer = keras.Input(shape=features_shape)
    inner_layer_1 = keras.layers.Dense(64, activation="selu")(input_layer)
    inner_layer_2 = keras.layers.Dense(32, activation="selu")(inner_layer_1)
    inner_layer_3 = keras.layers.Dense(16, activation="selu")(inner_layer_2)
    output_layer = keras.layers.Dense(target_shape)(inner_layer_3)

    model = keras.Model(inputs=input_layer, outputs=output_layer, name="NN_model")

    # Tune the learning rate for the optimizer, choose an optimal value [2]
    hp_learning_rate = hp.Float(
        "learning_rate", min_value=lr["min_value"], max_value=lr["max_value"]
    )

    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Nadam(learning_rate=hp_learning_rate),
        metrics=metrics,
    )

    return model

In [6]:
# we set a fixed random state for reproducibility and teaching purposes
tuner = kt.GridSearch(
    hypermodel=model_builder,
    objective=metrics,
    max_trials=max_trials,
    tune_new_entries=True,
    allow_new_entries=True,
    seed=seed,
    project_name="KerasTuner",
    # executions_per_trial = 10
)

# set an early stopping
callbacks = [keras.callbacks.EarlyStopping(monitor=monitor, patience=patience)]

# finally tune hyperpararmeters using the 10 percent of train data for
# validation
tuner.search(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=callbacks,
)

# get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)

# train the model again
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=callbacks,
)

# save the model
model.save("../models/model__Inverse_design_with_NN_Keras.keras")

Reloading Tuner from ./KerasTuner/tuner0.json
Epoch 1/2000


2024-11-26 16:39:24.338533: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3223 - loss: 0.5956 - val_accuracy: 0.3222 - val_loss: 0.4804
Epoch 2/2000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3761 - loss: 0.3913 - val_accuracy: 0.3889 - val_loss: 0.4095
Epoch 3/2000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.3816 - loss: 0.3473 - val_accuracy: 0.3667 - val_loss: 0.3977
Epoch 4/2000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4224 - loss: 0.3131 - val_accuracy: 0.3667 - val_loss: 0.3726
Epoch 5/2000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.4217 - loss: 0.2791 - val_accuracy: 0.3778 - val_loss: 0.3355
Epoch 6/2000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.4344 - loss: 0.2690 - val_accuracy: 0.3778 - val_loss: 0.3280
Epoch 7/2000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4354 - loss: 0.2602 - val_accuracy: 0.3778 - val_loss: 0.3366
Epoch 8/2000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - accuracy: 0.4383 - loss: 0.2525 - val_accuracy: 0.3778 

#### Alternatively, load a saved model

Load saved model and use it for the inverse design.

In [7]:
model = keras.saving.load_model("../models/model__Inverse_design_with_NN_Keras.keras")

## Inverse design

First, define the loss which determine the difference between our target and the output of the network as a function of the inputs of the network, i.e., define the objective function to be minimized. 

We select the Mean Squared Error.

In [8]:
def mse_loss(NN_input: np.array, model: keras.Model, target: float) -> float:
    """Obtain the Mean Squared Error between an ANN output and a given target.

    Intended for a single NN input, not a batch of inputs.
    """
    if len(NN_input.shape) > 1:
        raise ValueError("Function intended just for a single NN input. Terminated.")

    # add an aditional dimension to `NN_input` to add the batch size dimension,
    # which is just one
    NN_input_batched = np.expand_dims(NN_input, axis=0)

    # obtain the NN output using the model
    NN_output = model(NN_input_batched)

    # debug:
    # print(NN_input, NN_output)
    return (target - NN_output) ** 2

Then, specify the boundaries of each input (due to selected solver [4]) and store several radom initializations for the initial guess of  best inputs, so we assure the global minimum can be reached.

In [9]:
n_initializations = 10
int_features_shape = features_shape[0]  # indexing specific of this example

# common bound for all inputs of the ANN
input_bounds = [[lower_bound, higher_bound]] * int_features_shape

# random initializations of input parameters
init_guesses = np_rng.uniform(
    low=lower_bound, high=higher_bound, size=(n_initializations, int_features_shape)
)

Next, we select `3-point` as the method to compute the gradient method [4,5].

We finally select the target to retrieve and employ scipy minimization to reach the true optimal input for each initialization.

In [10]:
# select the target as the desired output decimal number for this example
target = 2
expected_opt = [1, 0]

min_loss = np.inf
for init_guess in init_guesses:
    # In order to tune the `minimize` function, `method` is an interesting
    # input, which can require to specify `bounds` input with a sequence of
    # (min, max) pairs for each element in x
    solver_output = minimize(
        mse_loss,
        init_guess,
        args=(model, target),
        jac="3-point",
        bounds=input_bounds,
    )
    print(f"\nSolver optimization sucsess: {solver_output.success}")
    if not solver_output.success:
        print(f"Cause of termination: {solver_output.message}")

    # analyze obtained optimized inputs
    print(
        f"\nGuess:{init_guess} -> {np.round(init_guess, 0)}",
        f"\nExpected: {expected_opt}"
        f"\nObtained: {solver_output.x} -> {np.round(solver_output.x, 0)}",
        f"\nloss: {solver_output.fun}",
    )

    if solver_output.fun < min_loss:
        min_loss = solver_output.fun
        best_solver = solver_output

print("\n\nRun with the minimum loss.")
print(
    f"\nExpected: {expected_opt}"
    f"\nObtained: {best_solver.x} -> {np.round(best_solver.x, 0)}",
    f"\nloss: {best_solver.fun}",
)


Solver optimization sucsess: True

Guess:[0.97728107 0.06004126] -> [1. 0.] 
Expected: [1, 0]
Obtained: [1.         0.08033884] -> [1. 0.] 
loss: 0.003503079293295741

Solver optimization sucsess: True

Guess:[0.91790611 0.27951045] -> [1. 0.] 
Expected: [1, 0]
Obtained: [0.99997211 0.07406613] -> [1. 0.] 
loss: 0.003489630063995719

Solver optimization sucsess: True

Guess:[0.0778871  0.62464581] -> [0. 1.] 
Expected: [1, 0]
Obtained: [0.08435209 0.67324169] -> [0. 1.] 
loss: 0.9626361727714539

Solver optimization sucsess: True

Guess:[0.89623647 0.3664157 ] -> [1. 0.] 
Expected: [1, 0]
Obtained: [0.9999837  0.07323474] -> [1. 0.] 
loss: 0.0034897285513579845

Solver optimization sucsess: True

Guess:[0.3466209  0.26236952] -> [0. 0.] 
Expected: [1, 0]
Obtained: [0.         0.06029409] -> [0. 0.] 
loss: 3.2413063049316406

Solver optimization sucsess: True

Guess:[0.31124719 0.06753173] -> [0. 0.] 
Expected: [1, 0]
Obtained: [2.94486230e-05 6.43047659e-02] -> [0. 0.] 
loss: 3.241002

From the results, we can see different initial guesses lead to different final results. 

Although it's outside the scope of this lesson since the behavior of the minimization will depend on the function to minimize, i.e., in the last term, of the NN under study, looking at the outputs after uncommenting `debug` lines this could be due to the small steps taken by the optimizer. 

Therefore, it is important to start from a good guess or several and sparse initializations in order to find the input of whose output is closest to the target.

#### References

.. [1] In notebook folder, [`Training_NN_Keras.ipynb`](https://github.com/Eva-ortiz/ML_hodgepodge/blob/main/notebook/Training_NN_Keras.ipynb)

.. [2] In notebook folder, [`Hp_optimization_NN_Keras.ipynb`](https://github.com/Eva-ortiz/ML_hodgepodge/blob/main/notebook/Training_NN_Keras.ipynb)

.. [3] https://numpy.org/doc/stable/reference/random/index.html 

.. [4] https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html 

.. [5] https://www.vaia.com/en-us/textbooks/math/numerical-analysis-9-edition/chapter-4/problem-6-use-the-most-accurate-three-point-formula-to-deter/ 

#### Acknowledgements

[Juan José García Esteban](https://github.com/jjgarciae?tab=overview&from=2024-11-01&to=2024-11-26) - for first teaching me the basic usage of this process